### Gradients:

* We have $X_t, X_{t-1} \in \mathbb{R}^{I \times K}$, $Y_t \in \mathbb{R}^{I \times J}$.

* 
\begin{align}
\nabla_c \log p_\theta(X_t \mid X_{t-1}) 
=
\nabla_c\left \{ \frac{-1}{2 \sigma^2} \| c + \phi X_{t-1} - X_t \|^2 \right \} 
& = 
\frac{-1}{\sigma^2} \left \{ c(IK)^2 + \langle \mathbf{1} \mathbf{1}^\top, \phi X_{t-1} - X_t \rangle \right \} 
\\ 
\nabla_\phi \log p_\theta(X_t \mid X_{t-1}) 
=
\nabla_\phi \left \{ \frac{-1}{2 \sigma^2} \| c + \phi X_{t-1} - X_t \|^2 \right \} 
& = 
\frac{-1}{\sigma^2} \left \{ \phi \|X_{t-1}\|^2 - \langle X_{t-1}, c-X_t \rangle \right \} 
\\
\nabla_{\sigma^2} \log p_\theta(X_t \mid X_{t-1}) 
=
\nabla_{\sigma^2} \left \{ \frac{-1}{2 \sigma^2} \| c + \phi X_{t-1} - X_t \|^2 \right \} 
& = 
\frac{1}{2 \sigma^4} \| c + \phi X_{t-1} - X_t \|^2.
\end{align} 

#### Fix parameters of latent chain.

In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp, copy
from pylab import plot, show, legend

from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [16]:
T = 100
I = 10  # number of locations
J = 5  # number of species
K = 3   # number of latent factors

In [21]:
lmbda = npr.randn(J,K)
alpha = npr.randn(J)
c = 0
phi = 0.5
logsigmasq = 0
x_0 = npr.randn(I,K)

theta = [alpha, lmbda, c, phi, logsigmasq]
Y, X = simulate_data(x_0, T, J, theta)

In [22]:
n_particles = 200
alpha_grad, lmbda_grad, c_grad, phi_grad, logsigmasq_grad, logNC = \
get_grads(Y, x_0, n_particles, theta, Tmax=T, Imax=I, Jmax=J, Kmax=K, rep=200)
print("Variance of log-likelihood       :", np.var(logNC,0))
print("Mean of gradient w.r.t. alpha    :", np.mean(alpha_grad,0))
print("Variance of gradient w.r.t. alpha:", np.var(alpha_grad,0))

Variance of log-likelihood       : [187.75966171 328.34581586]
Mean of gradient w.r.t. alpha    : [[-6.97219925  1.54780475 16.83232774  5.15807709  2.32185034]
 [-5.73731265  1.26312597 13.86929473  4.56617281 -0.27011305]]
Variance of gradient w.r.t. alpha: [[ 34.60417049  34.50239485  12.33175435  22.51251607  33.34634857]
 [111.65936333 129.66963645  31.288103    62.99699686 126.85868001]]


In [ ]:
n_mcmc = 10_000
n_particles = 1000
tau = 1e-2*np.ones(5)
update = [1, *np.zeros(4)]
theta_mala, _ = pMCMC_MALA_blockPF(Y, x_0, n_particles, theta, n_mcmc, tau, update, power=1)

/home/postdoc/dsen/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)
 23%|██▎       | 2284/10000 [1:10:00<4:14:26,  1.98s/it]

In [ ]:
plt.plot(theta_mala[0])
plt.grid(True);

In [ ]:
n_mcmc = 10_000
n_particles = 1000
eps = 1.5e-1
scale = [eps*np.ones(J), eps*np.ones((J,K)), eps, eps, eps]
theta_0 = theta

In [ ]:
theta_chain, scale, accept_probs = pMCMC_RW(x_0, Y, theta_0, n_particles, n_mcmc, scale, power=1, adapt=True)

In [ ]:
plt.plot(theta_chain[0])
plt.grid(True);

In [12]:
np.std(theta_chain[0],0)

array([0.20470775, 0.25031098, 0.18472617])